In [9]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data,housing.target)
X_train,X_valid,y_train,y_valid = train_test_split(X_train_full,y_train_full)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_train_A,X_train_B = X_train[:,:5],X_train[:,2:]
X_valid_A,X_valid_B = X_valid[:,:5],X_valid[:,2:]
X_test_A,X_test_B = X_test[:,:5],X_test[:,2:]
X_new_A,X_new_B = X_test_A[:3],X_test_B[:3]
X_valid = scaler.fit_transform(X_valid)
X_test = scaler.fit_transform(X_test)

In [10]:
input_ = keras.layers.Input(shape = X_train.shape[1:])
hidden1 = keras.layers.Dense(30,activation="relu")(input_)
hidden2 = keras.layers.Dense(30,activation="relu")(hidden1)
concat = keras.layers.Concatenate()([input_,hidden2])
output = keras.layers.Dense(1)(concat)
model = keras.Model(inputs = [input_],outputs = [output])


In [13]:
class WideAndDeepModel(keras.Model):
    def __init__(self,units=30,activation="relu",**kwargs):
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(units,activation=activation)
        self.hidden2 = keras.layers.Dense(units,activation=activation)
        self.main_output = keras.layers.Dense(1)
        self.aux_output = keras.layers.Dense(1)
    def call(self,inputs):
        input_A,input_B = inputs
        hidden1 = self.hidden1(input_B)
        hidden2 = self.hidden2(hidden1)
        concat = keras.layers.concatenate([input_A,hidden2])
        main_output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return main_output,aux_output
model = WideAndDeepModel()

In [15]:
model.compile(loss=["mse","mse"],loss_weights=[0.9,.1],optimizer="sgd")
history = model.fit([X_train_A,X_train_B],[y_train,y_train],epochs = 20,validation_data=([X_valid_A,X_valid_B],[y_valid,y_valid]))
y_pred_main,y_pred_aux = model.predict([X_new_A,X_new_B])
print(y_pred_main)

Epoch 1/20
363/363 [==============================] - 1s 2ms/step - loss: 0.3497 - output_1_loss: 0.3368 - output_2_loss: 0.4665 - val_loss: 7383.0093 - val_output_1_loss: 5166.5811 - val_output_2_loss: 27330.8477
Epoch 2/20
363/363 [==============================] - 0s 1ms/step - loss: 0.3476 - output_1_loss: 0.3349 - output_2_loss: 0.4619 - val_loss: 9088.8691 - val_output_1_loss: 7126.6987 - val_output_2_loss: 26748.4102
Epoch 3/20
363/363 [==============================] - 0s 1ms/step - loss: 0.3520 - output_1_loss: 0.3394 - output_2_loss: 0.4651 - val_loss: 4323.2500 - val_output_1_loss: 2400.5261 - val_output_2_loss: 21627.7598
Epoch 4/20
363/363 [==============================] - 0s 1ms/step - loss: 0.3485 - output_1_loss: 0.3368 - output_2_loss: 0.4541 - val_loss: 4828.9595 - val_output_1_loss: 3321.9370 - val_output_2_loss: 18392.1680
Epoch 5/20
363/363 [==============================] - 0s 1ms/step - loss: 0.3444 - output_1_loss: 0.3325 - output_2_loss: 0.4508 - val_loss: 588

In [25]:
total_loss,main_loss,aux_loss=model.evaluate([X_test_A,X_test_B],[y_test,y_test])

162/162 [==============================] - 0s 809us/step - loss: 10803.2773 - output_1_loss: 9838.3867 - output_2_loss: 19487.3184
